## AUs and Smile Types were analyzed with Gender 
### Comparison was done as follows using mann-whitney U
Male True vs. Male Bluff 

Female True vs. Female Bluff

Male True vs. Female True

Female Bluff vs. Male Bluff

Male True vs. Female Bluff

Female True vs. Male Bluff 

In [51]:
#datafile = 'all_frames.pkl.xz' # FG/UBICOMP data N=151
datafile_km = '../data/all_frames_wclust.pkl.xz' # with AU6_AU12 clusters
datafile_bmm = '../data/all_frames_clust.bmm.pkl.xz' # bmm with 5 clusters
datafile_bmm9 = '../data/all_frames_clust.bmm9.pkl.xz' # bmm clusters
datafile = '../data/all_frames_au.pkl.xz' # all AUS

CONFIDENCE_TOL = 0.90 # only use data with conf > this

#-----------------

import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,.4f}'.format
from IPython.display import display
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
# import in scripts dir
import sys
sys.path.append('../scripts')
import compare 

cluster_name2i = {'duchenne':0,'neutral':1,'strong duchenne':2,'6 only':3,'polite':4}
cluster_name2i_bmm9 = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8}

def load_cluster_data(datafile):
    print('\n...loading cluster data: ', datafile)
    if 'pkl' in datafile:
        df = pd.read_pickle(datafile)
    else:
        df = pd.read_csv(datafile, skipinitialspace=True) 
    
    # confidence of .90 causes nfiles 302-->298
    df = df[df['confidence'] >= CONFIDENCE_TOL]


    if 'bmm9' in datafile:
        for cluster_name in cluster_name2i_bmm9.keys():
            df[cluster_name] = (df['AU06_AU12_cluster'] == cluster_name2i_bmm9[cluster_name]).astype(int)
    elif 'au' in datafile:
        AUS = [x for x in df.columns if '_r' in x]
    else:
        for cluster_name in cluster_name2i.keys():
            df[cluster_name] = (df['AU06_AU12_cluster'] == cluster_name2i[cluster_name]).astype(int)

    print('n:',df.shape[0])
    print('# files = ', df['Filename'].nunique())

    print('\ncolumns: ')
    for c in df.columns:
        print(c,end=',')
    print('-------')
    return df
        
df_km = load_cluster_data(datafile_km)
df_bmm = load_cluster_data(datafile_bmm)
df_bmm9 = load_cluster_data(datafile_bmm9)
dfAU= load_cluster_data(datafile)

print("loads data of the gender")
genderdat = '../data/old_data_gender.csv'
Gen_df=pd.read_csv(genderdat)


...loading cluster data:  ../data/all_frames_wclust.pkl.xz
n: 1246319
# files =  298

columns: 
Filename,filetype,segment,timestamp,confidence,success,pose_Tx,pose_Ty,pose_Tz,pose_Rx,pose_Ry,pose_Rz,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r,AU01_c,AU02_c,AU04_c,AU05_c,AU06_c,AU07_c,AU09_c,AU10_c,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,AU06_AU12_cluster,polite,6 only,neutral,duchenne,strong duchenne,-------

...loading cluster data:  ../data/all_frames_clust.bmm.pkl.xz
n: 1246319
# files =  298

columns: 
Filename,filetype,confidence,segment,timestamp,AU06_AU12_cluster,polite,6 only,neutral,duchenne,strong duchenne,-------

...loading cluster data:  ../data/all_frames_clust.bmm9.pkl.xz
n: 1246319
# files =  298

columns: 
Filename,filetype,confidence,segment,timestamp,AU06_AU12_cluster,7,1,8,6,5,3,4,2,0,-------

...loading cluster data:  ../data/all_frames_au.pkl.xz
n: 1246319
# f

#### Looking at Basic Stats of each filetype

In [52]:
# cluster statistics
print('Average cluster dist over all frames')
print('K-MEANS')
g = df_km.groupby('filetype')
cluster_dist = g[list(cluster_name2i.keys())].mean()
display(cluster_dist)

print('\n\nBETA MIXTURE bmm5')
g = df_bmm.groupby('filetype')
cluster_dist = g[list(cluster_name2i.keys())].mean()
display(cluster_dist)

print('\n\nBETA MIXTURE bmm9')
g = df_bmm9.groupby('filetype')
cluster_dist = g[list(cluster_name2i_bmm9.keys())].mean()
display(cluster_dist)

Average cluster dist over all frames
K-MEANS


,polite,6 only,neutral,duchenne,strong duchenne
filetype,,,,,
I-B,0.2371,0.1632,0.3701,0.1715,0.0581
I-T,0.1599,0.1842,0.4403,0.1568,0.0588
W-B,0.2282,0.1268,0.4385,0.1339,0.0725
W-T,0.2307,0.1785,0.4167,0.1299,0.0442




BETA MIXTURE bmm5


,polite,6 only,neutral,duchenne,strong duchenne
filetype,,,,,
I-B,0.1620,0.0565,0.1426,0.4395,0.1994
I-T,0.1060,0.0893,0.1909,0.4179,0.1959
W-B,0.1780,0.0726,0.1695,0.4026,0.1774
W-T,0.1580,0.0801,0.1374,0.4777,0.1468




BETA MIXTURE bmm9


,7,1,8,6,5,3,4,2,0
filetype,,,,,,,,,
I-B,0.2113,0.0643,0.0235,0.0520,0.2273,0.1426,0.0812,0.1527,0.0450
I-T,0.1999,0.0644,0.0245,0.0816,0.2129,0.1909,0.0604,0.0994,0.0662
W-B,0.1490,0.0649,0.0370,0.0669,0.2060,0.1694,0.0910,0.1670,0.0488
W-T,0.1610,0.0506,0.0190,0.0721,0.2557,0.1374,0.0851,0.1474,0.0719


In [53]:
## Implements Mann Whitney U and t-test from a given tupe of dataframe s 
import unittest
import logging
import scipy.stats as stats
def calc_stats(s, print_=False, fig_=None):
        """ returns: X.mean,
                     Y.mean,
                     t-test_p, 
                     Mann-Whitney test_p, 
                     Cohens_d """
        
        t,t_test_p = stats.ttest_ind(s[0], s[1], axis=0, equal_var=False)
        try:
            mw,mw_p = stats.mannwhitneyu(s[0], s[1])
        except ValueError:
            mw,mw_p = np.NAN,np.NAN

        n_tot = len(s[0]) + len(s[1])
        x_var = s[0].var(ddof=0)
        y_var = s[1].var(ddof=0)
        pooled_var = (len(s[0])*x_var + len(s[1])*y_var) / n_tot
        
        if(pooled_var ==0):
            cohens_d = np.nan
        else:
            cohens_d = (s[0].mean() - s[1].mean()) / np.sqrt(pooled_var)
            

        my_stats = s[0].mean(), s[1].mean(), t_test_p, mw_p, cohens_d
        if print_:
            s.print_stats(my_stats)
        return my_stats

In [80]:
# df_bmm['root']=df_bmm['Filename'].str.split('-W').str[0]
# dfGenbmm=pd.merge(df_bmm, Gen_df, how = 'left', left_on = 'root', right_on = 'root')
df_bmm1= df_bmm[df_bmm['segment']=='S2']
df_bmm_gr1= group_Gen(df_bmm1,'W')#pd.DataFrame(dfGenbmm.groupby(['root']).mean().dropna())
a = df_bmm_gr1[(df_bmm_gr1['witness_is_male'] == 1) & (df_bmm_gr1['describer_job_is_truth'] == 1)]['polite']
b = df_bmm_gr1[(df_bmm_gr1['witness_is_male'] == 1) & (df_bmm_gr1['describer_job_is_truth'] == 0)]['polite']
s, p = stats.mannwhitneyu(a,b)

calc_stats((a,b))

(0.13303604140919245,
 0.27774763390823015,
 0.0057904205983919524,
 0.0311572052518404,
 -0.65601259487374497)

In [82]:
# typewi : type can be interrogator or witness (W or I)
def group_Gen(df,typewi):
    df['root']=df['Filename'].str.split('-'+typewi).str[0]
    dfGen=pd.merge(df,Gen_df, how='inner',left_on='root',right_on= 'root')
    dfgr= pd.DataFrame(dfGen.groupby(['root']).mean().dropna()).reset_index()
    
    return dfgr

def split(df):
    mt = df[(df['witness_is_male'] == 1) & (df['describer_job_is_truth'] == 1)]
    ft = df[(df['witness_is_male'] == 0) & (df['describer_job_is_truth'] == 1)]
    mb = df[(df['witness_is_male'] == 1) & (df['describer_job_is_truth'] == 0)]
    fb = df[(df['witness_is_male'] == 0) & (df['describer_job_is_truth'] == 0)]
    
    dfpartition = {'male true':mt,'female true':ft,'male bluff':mb,'female bluff':fb}
    return dfpartition

def do_stats(d1,d2,cluster_names):
    statss = []
    for c in cluster_names:
        comp = (d1[c],d2[c])
        statss.append(np.array(calc_stats(comp)))
    return statss

def testall(df,cluster_names):

    for person in ['W','I']:
        print (person)
        data_gender=group_Gen(df,person)
        dfdict = split(data_gender)
        dkeys = list(dfdict.keys())
        x=0
        for k,v in dfdict.items():
            for i in range(x,len(dkeys)):
                if k!= dkeys[i]:
                   print (k,' vs. ',dkeys[i])
                   g1 = dfdict[k]
                   g2 = dfdict[dkeys[i]] 
                   statss=do_stats(g1,g2,cluster_names)
                   df_cluster = pd.DataFrame(np.array(statss))
                   df_cluster.columns=[k+' mean',dkeys[i]+' mean','tt p-val','MW p-val','d']
                   df_cluster.index = cluster_names
                   display(df_cluster)
                   
            x=x+1


In [83]:
print('KMM 5')                
df_km = df_km[(df_km['segment'] == 'S2')]
cluster_names = list(cluster_name2i.keys())
testall(df_km,cluster_names)

KMM 5
W
male bluff  vs.  female bluff


,male bluff mean,female bluff mean,tt p-val,MW p-val,d
polite,0.3101,0.2258,0.1828,0.0875,0.3203
6 only,0.0820,0.2490,0.0078,0.0060,-0.7013
neutral,0.3917,0.3378,0.5192,0.2917,0.1551
duchenne,0.1504,0.1194,0.4221,0.4082,0.1921
strong duchenne,0.0657,0.0680,0.9356,0.3575,-0.0191


male bluff  vs.  male true


,male bluff mean,male true mean,tt p-val,MW p-val,d
polite,0.3101,0.2498,0.3223,0.1908,0.2323
6 only,0.0820,0.1407,0.1597,0.0051,-0.3355
neutral,0.3917,0.4599,0.3836,0.1726,-0.2044
duchenne,0.1504,0.1338,0.6988,0.2825,0.0915
strong duchenne,0.0657,0.0157,0.0305,0.0157,0.5058


male bluff  vs.  female true


...

,female bluff mean,male true mean,tt p-val,MW p-val,d
polite,0.2673,0.3033,0.5879,0.1983,-0.1343
6 only,0.2122,0.1597,0.3651,0.2658,0.2258
neutral,0.2273,0.3393,0.1265,0.0688,-0.3796
duchenne,0.2312,0.1522,0.1260,0.0594,0.3862
strong duchenne,0.0620,0.0455,0.5878,0.1066,0.1358


female bluff  vs.  female true


,female bluff mean,female true mean,tt p-val,MW p-val,d
polite,0.2673,0.1889,0.2417,0.1644,0.2934
6 only,0.2122,0.1831,0.6527,0.3736,0.1120
neutral,0.2273,0.4629,0.0071,0.0207,-0.6880
duchenne,0.2312,0.1330,0.0746,0.0089,0.4511
strong duchenne,0.0620,0.0320,0.3204,0.0188,0.2504


male true  vs.  female true


,male true mean,female true mean,tt p-val,MW p-val,d
polite,0.3033,0.1889,0.0784,0.0121,0.4367
6 only,0.1597,0.1831,0.6939,0.4879,-0.0969
neutral,0.3393,0.4629,0.1643,0.1690,-0.3450
duchenne,0.1522,0.1330,0.6700,0.1514,0.1049
strong duchenne,0.0455,0.0320,0.5591,0.2434,0.1433


In [84]:
print('BMM 5')                
df_bmm = df_bmm[(df_bmm['segment'] == 'S2')]
cluster_names = list(cluster_name2i.keys())
testall(df_bmm,cluster_names)

BMM 5
W
male bluff  vs.  female bluff


,male bluff mean,female bluff mean,tt p-val,MW p-val,d
polite,0.2777,0.1273,0.0074,0.0072,0.6417
6 only,0.0445,0.0756,0.3579,0.0924,-0.2330
neutral,0.0978,0.1413,0.4163,0.3467,-0.2034
duchenne,0.3986,0.4894,0.1715,0.1155,-0.3397
strong duchenne,0.1814,0.1663,0.7872,0.3251,0.0646


male bluff  vs.  male true


,male bluff mean,male true mean,tt p-val,MW p-val,d
polite,0.2777,0.1330,0.0058,0.0312,0.6560
6 only,0.0445,0.0616,0.4442,0.0141,-0.1804
neutral,0.0978,0.1473,0.3219,0.0808,-0.2374
duchenne,0.3986,0.5464,0.0129,0.0059,-0.6008
strong duchenne,0.1814,0.1118,0.1620,0.1413,0.3271


male bluff  vs.  female true


...

,female bluff mean,male true mean,tt p-val,MW p-val,d
polite,0.1324,0.1026,0.4489,0.1939,0.1906
6 only,0.0574,0.0722,0.7268,0.0939,-0.0862
neutral,0.0419,0.0967,0.1993,0.1839,-0.3151
duchenne,0.5316,0.5652,0.6255,0.2946,-0.1208
strong duchenne,0.2368,0.1634,0.2165,0.1017,0.3100


female bluff  vs.  female true


,female bluff mean,female true mean,tt p-val,MW p-val,d
polite,0.1324,0.1510,0.7252,0.4763,-0.0874
6 only,0.0574,0.0499,0.8183,0.1293,0.0577
neutral,0.0419,0.1674,0.0228,0.0593,-0.5788
duchenne,0.5316,0.4882,0.5483,0.2633,0.1495
strong duchenne,0.2368,0.1435,0.1294,0.0137,0.3824


male true  vs.  female true


,male true mean,female true mean,tt p-val,MW p-val,d
polite,0.1026,0.1510,0.2877,0.1773,-0.2651
6 only,0.0722,0.0499,0.5227,0.4826,0.1559
neutral,0.0967,0.1674,0.2513,0.2344,-0.2841
duchenne,0.5652,0.4882,0.2830,0.1387,0.2648
strong duchenne,0.1634,0.1435,0.7036,0.2291,0.0935


In [85]:
print('BMM 9')                
df_bmm9 = df_bmm9[(df_bmm9['segment'] == 'S2')].copy()
cluster_namesbm9 = list(cluster_name2i_bmm9.keys())
testall(df_bmm9,cluster_namesbm9)

BMM 9
W
male bluff  vs.  female bluff


,male bluff mean,female bluff mean,tt p-val,MW p-val,d
7,0.1604,0.1771,0.7128,0.1978,-0.0888
1,0.0738,0.0607,0.6131,0.4269,0.1210
8,0.0216,0.0308,0.5205,0.1348,-0.1521
6,0.0392,0.0715,0.3202,0.1126,-0.2528
5,0.1908,0.2658,0.0840,0.0840,-0.4349
3,0.0978,0.1413,0.4164,0.3467,-0.2034
4,0.1065,0.0881,0.4381,0.1427,0.1864
2,0.2626,0.1157,0.0066,0.0066,0.6499
0,0.0472,0.0491,0.9159,0.0608,-0.0246


male bluff  vs.  male true


,male bluff mean,male true mean,tt p-val,MW p-val,d
7,0.1604,0.1649,0.9202,0.3729,-0.0236
1,0.0738,0.0325,0.0630,0.0436,0.4354
8,0.0216,0.0036,0.1018,0.2226,0.3775
6,0.0392,0.0519,0.5313,0.0107,-0.1473
5,0.1908,0.2861,0.0179,0.0139,-0.5759
3,0.0978,0.1472,0.3222,0.0808,-0.2373
4,0.1065,0.1036,0.9026,0.4810,0.0288
2,0.2626,0.1213,0.0055,0.0319,0.6603
0,0.0472,0.0889,0.0602,0.0078,-0.4490


male bluff  vs.  female true


...

,female bluff mean,male true mean,tt p-val,MW p-val,d
7,0.3047,0.1812,0.0551,0.0278,0.4874
1,0.0666,0.0509,0.5557,0.1501,0.1474
8,0.0209,0.0162,0.6900,0.0950,0.0993
6,0.0515,0.0673,0.7017,0.1210,-0.0944
5,0.2547,0.3276,0.1713,0.0895,-0.3405
3,0.0418,0.0967,0.1991,0.1839,-0.3153
4,0.0739,0.1085,0.1800,0.0810,-0.3313
2,0.1236,0.0920,0.3984,0.2213,0.2128
0,0.0622,0.0596,0.9202,0.2176,0.0250


female bluff  vs.  female true


,female bluff mean,female true mean,tt p-val,MW p-val,d
7,0.3047,0.1988,0.1282,0.0093,0.3833
1,0.0666,0.0380,0.2938,0.0199,0.2639
8,0.0209,0.0129,0.4728,0.0656,0.1804
6,0.0515,0.0424,0.7683,0.1052,0.0742
5,0.2547,0.2324,0.6831,0.2674,0.1017
3,0.0418,0.1672,0.0229,0.0593,-0.5783
4,0.0739,0.0736,0.9918,0.4820,0.0026
2,0.1236,0.1416,0.7241,0.4393,-0.0877
0,0.0622,0.0931,0.4244,0.1341,-0.1984


male true  vs.  female true


,male true mean,female true mean,tt p-val,MW p-val,d
7,0.1812,0.1988,0.7576,0.2601,-0.0760
1,0.0509,0.0380,0.5608,0.1607,0.1429
8,0.0162,0.0129,0.7300,0.4090,0.0844
6,0.0673,0.0424,0.4602,0.4425,0.1802
5,0.3276,0.2324,0.0826,0.0242,0.4309
3,0.0967,0.1672,0.2526,0.2344,-0.2833
4,0.1085,0.0736,0.1947,0.0514,0.3194
2,0.0920,0.1416,0.2584,0.2284,-0.2822
0,0.0596,0.0931,0.3512,0.3203,-0.2322


In [86]:
print('SEGMENT = S2 ONLY - AUs')
df_AU = dfAU[(dfAU['segment'] == 'S2')]
cluster_names = [x for x in dfAU.columns if '_r' in x]
testall(df_AU,cluster_names)

SEGMENT = S2 ONLY - AUs
W


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


male bluff  vs.  female bluff


,male bluff mean,female bluff mean,tt p-val,MW p-val,d
AU01_r,0.3774,0.3575,0.7661,0.4955,0.0705
AU02_r,0.1526,0.1343,0.5428,0.2112,0.1472
AU04_r,0.4140,0.5861,0.2785,0.2671,-0.2687
AU05_r,0.1228,0.1137,0.7073,0.3379,0.0878
AU06_r,0.4892,0.6366,0.1863,0.0857,-0.3200
AU07_r,0.8665,0.8579,0.9596,0.4595,0.0122
AU09_r,0.1303,0.1864,0.2270,0.1008,-0.3044
AU10_r,0.6847,0.8214,0.2924,0.1177,-0.2567
AU12_r,0.8005,0.7171,0.5049,0.2860,0.1587
AU14_r,0.7203,0.7325,0.9292,0.3175,-0.0217


male bluff  vs.  male true


...

,female bluff mean,male true mean,tt p-val,MW p-val,d
AU01_r,0.3236,0.4065,0.3007,0.0755,-0.2569
AU02_r,0.1143,0.1323,0.5815,0.0248,-0.1383
AU04_r,0.3518,0.6004,0.0365,0.0032,-0.5277
AU05_r,0.0511,0.0714,0.1928,0.0513,-0.3230
AU06_r,0.7363,0.5736,0.1627,0.0885,0.3509
AU07_r,0.7468,0.6915,0.7314,0.3842,0.0853
AU09_r,0.1097,0.0935,0.4978,0.1219,0.1678
AU10_r,0.8548,0.7359,0.3816,0.2738,0.2187
AU12_r,0.9187,0.7629,0.2182,0.1122,0.3054
AU14_r,1.1431,0.9101,0.1250,0.0738,0.3836


female bluff  vs.  female true


,female bluff mean,female true mean,tt p-val,MW p-val,d
AU01_r,0.3236,0.2930,0.6588,0.4576,0.1107
AU02_r,0.1143,0.1109,0.9232,0.3372,0.0241
AU04_r,0.3518,0.5011,0.2507,0.0950,-0.2871
AU05_r,0.0511,0.0411,0.4279,0.3058,0.1992
AU06_r,0.7363,0.5443,0.1335,0.0350,0.3773
AU07_r,0.7468,0.5097,0.1193,0.0919,0.3941
AU09_r,0.1097,0.0854,0.2354,0.1816,0.2994
AU10_r,0.8548,0.7493,0.4486,0.3014,0.1897
AU12_r,0.9187,0.6024,0.0121,0.0033,0.6403
AU14_r,1.1431,0.8640,0.0812,0.0340,0.4394


male true  vs.  female true


,male true mean,female true mean,tt p-val,MW p-val,d
AU01_r,0.4065,0.2930,0.0967,0.0757,0.4101
AU02_r,0.1323,0.1109,0.4464,0.0383,0.1879
AU04_r,0.6004,0.5011,0.4188,0.1071,0.1994
AU05_r,0.0714,0.0411,0.0265,0.0098,0.5531
AU06_r,0.5736,0.5443,0.7909,0.1761,0.0653
AU07_r,0.6915,0.5097,0.1854,0.1116,0.3261
AU09_r,0.0935,0.0854,0.6879,0.4832,0.0981
AU10_r,0.7359,0.7493,0.9108,0.4546,-0.0275
AU12_r,0.7629,0.6024,0.2061,0.0866,0.3117
AU14_r,0.9101,0.8640,0.7599,0.3177,0.0751
